In [23]:
import pandas as pd
pets = pd.read_csv('pets.csv')
pets

,Owner,Animal,Legs,Gender,Ownership_date
0,Alfred,Dog,4,Male,14/5/21
1,Carl,Dog,4,Female,11/6/19
2,John,Cat,4,Female,21/7/22
3,Gabriel,Canari,2,NaN,3/1/23
4,Harold,Hamster,4,Male,14/5/16
5,Anna,Iguana,4,NaN,29/8/22
6,Brigitte,Cat,4,Female,13/4/20
7,Christine,Parrot,2,Male,15/9/19
8,Eva,Snake,0,NaN,6/2/21
9,Greta,Parrot,2,Male,17/7/18


In [24]:
pets[(pets['Animal'] == 'Dog') & (pets['Gender'] == 'Female')]

,Owner,Animal,Legs,Gender,Ownership_date
1,Carl,Dog,4,Female,11/6/19
10,Agnes,Dog,4,Female,11/11/22


# Part 2 - Retrieval

* Retrieval is the ability display (and assign to a new dataframe) just parts of the data set: a **subset** of the rows and/or a **subset** of the columns.
* In Pandas, each *column* and each row have *labels* (i.e. names)
* If labels aren't provided when creating the dataframe, the labels will be serial numbers 0, 1, 2, etc.
* Let's discuss retrieval by column/row name
* The row names are called the **index**.



## 2A. The `.loc[row_labels, column_labels]` special method
* the row and column labels can be either *string*, *list* or *slice* (colon notation)



* Let's see a few common use cases:
    1. **Extract a single column**
    1. **Extract a single row**
    1. **Extract a range of columns**
    1. Specific columns in specific order
    1. Combinations
    

### Task 1 - Extract a single column
* How do we get the `Animal` column from the dataset?

In [25]:
# retrieve rows: all, columns: 'Animal'
pets.loc[:, 'Animal']

0         Dog
1         Dog
2         Cat
3      Canari
4     Hamster
5      Iguana
6         Cat
7      Parrot
8       Snake
9      Parrot
10        Dog
Name: Animal, dtype: object

### Task 2 - extract a single row
* We can place a `:` in the second argument
* ...or simply drop it.


In [26]:
pets.loc[3, :]

Owner             Gabriel
Animal             Canari
Legs                    2
Gender                NaN
Ownership_date     3/1/23
Name: 3, dtype: object

In [27]:
pets.loc[3]

Owner             Gabriel
Animal             Canari
Legs                    2
Gender                NaN
Ownership_date     3/1/23
Name: 3, dtype: object

#### &#x1F4A1; Why is the data displayed vertically when we extract a row? &#x1F4A1;
* Pandas has two main data types - `Series` and `DataFrame`
* Pandas **stores** each **column** of a data frame as a `Series`
* `.loc[3]` **created** a temporary `Series` object in order to display the data in the row
* a `Series` is always displayed vertically
* Therefore, the output looks vertical even for a row

## Task 3 - extract a range of columns
* Extract all columns from "Animal" to "Gender"


In [28]:
pets.loc[:, "Animal":"Gender"]

,Animal,Legs,Gender
0,Dog,4,Male
1,Dog,4,Female
2,Cat,4,Female
3,Canari,2,NaN
4,Hamster,4,Male
5,Iguana,4,NaN
6,Cat,4,Female
7,Parrot,2,Male
8,Snake,0,NaN
9,Parrot,2,Male


## Task 4 - extract specific columns in arbitrary order
* Let's display animal, ownership date and owner

In [29]:
pets.loc[:, ["Animal", "Ownership_date", "Owner"]]

,Animal,Ownership_date,Owner
0,Dog,14/5/21,Alfred
1,Dog,11/6/19,Carl
2,Cat,21/7/22,John
3,Canari,3/1/23,Gabriel
4,Hamster,14/5/16,Harold
5,Iguana,29/8/22,Anna
6,Cat,13/4/20,Brigitte
7,Parrot,15/9/19,Christine
8,Snake,6/2/21,Eva
9,Parrot,17/7/18,Greta


## Task 5 - combinations

* Display just the owner and ownership data of rows from 5 onward


In [30]:
pets.loc[5:, ["Owner", "Ownership_date"]]

,Owner,Ownership_date
5,Anna,29/8/22
6,Brigitte,13/4/20
7,Christine,15/9/19
8,Eva,6/2/21
9,Greta,17/7/18
10,Agnes,11/11/22


## Task 3 - Conditional row retrieval
* Task: Select just the rows where `Animal=='Dog'` and `Gender=='Female'`


### Method 1: `query`



In [31]:
pets.query("Animal == 'Dog' and Gender == 'Female'")

,Owner,Animal,Legs,Gender,Ownership_date
1,Carl,Dog,4,Female,11/6/19
10,Agnes,Dog,4,Female,11/11/22


####  The  `query`  string - rules
* Use Python expressions
* How to specify column names:
    * If they look like variable names - unquoted
    * If they have spaces etc. - enclosed in backticks: \`column name\`
* Variables are supported (prefix with `@`)


In [32]:
animal_type = 'Dog'
pets.query("Animal == @animal_type and Gender == 'Female'")

,Owner,Animal,Legs,Gender,Ownership_date
1,Carl,Dog,4,Female,11/6/19
10,Agnes,Dog,4,Female,11/11/22


### Method 2:  `loc[mask]`
* This method results in bulkier code, but is important to know and understand.
* Let's first understand the following Pandas mechanisms:
    * *mask* (also known as *logical indexing*)
    * *broadcast*
    * logical operations (i.e. and, or, not) between series
#### 1. Mask
* A mask is list or Series with boolean values (True/False)
* A mask is applied to a column
* The mask must be exactly the same length as the masked column or columns
* The mask "allows through" only values corresponding (by location) to True entries




In [33]:
# Example mask with 11 entries
# True entries have been "highlighted"

#         0       1    *2*    3     *4*     5    *6*     7     8       9     10
mask = [False, False, True, False, True,
        False, True, False, False, False, False]


# Let's apply the mask to the pets DataFrame:
pets.loc[mask]

,Owner,Animal,Legs,Gender,Ownership_date
2,John,Cat,4,Female,21/7/22
4,Harold,Hamster,4,Male,14/5/16
6,Brigitte,Cat,4,Female,13/4/20


#### 2. Broadcast
* As first step, let's say we want to retrieve all rows where `Animal` is Dog
* We need to compare *each element* in the `Animal` Series to the string `'Dog'` and return a boolean
* *Broadcast* spares us from writing a loop (or a list comprehension)

In [34]:
pets.Animal == 'Dog'

0      True
1      True
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10     True
Name: Animal, dtype: bool

* Pandas understood this expression as an **element by element comparison**
* This ability to "replicate" a single value across rows is known as *broadcast(


#### 3. Combine broadcast and logical indexing

#### 4. Logical operators between series
* Our second condition is `pets.Gender == 'Female'`
* At this point, we have two boolean series, one for each condition
* Can we use `and`?
   

In [ ]:
# Error!
# pets.Animal=='Dog' and pets.Gender=='Female'


# ---------------------------------------------------------------------------
# ValueError                                Traceback (most recent call last)
# Input In [106], in <cell line: 2>()
#       1 # Error!
# ----> 2 pets.Animal=='Dog' and pets.Gender=='Female'

# File ~/miniconda3/envs/base/lib/python3.9/site-packages/pandas/core/generic.py:1527, in NDFrame.__nonzero__(self)
#    1525 @final
#    1526 def __nonzero__(self):
# -> 1527     raise ValueError(
#    1528         f"The truth value of a {type(self).__name__} is ambiguous. "
#    1529         "Use a.empty, a.bool(), a.item(), a.any() or a.all()."
#    1530     )

# ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [36]:
# Correct!
(pets.Animal == 'Dog') & (pets.Gender == 'Female')

0     False
1      True
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10     True
dtype: bool

Rules:
* Operators
    * `and` &rArr; `&`  (ampersand)
    * `or`  &rArr; `|`  (pipe)
    * `not` &rArr; `~`  (tilde)
* **enclose each condition in parentheses**

#### 5. Putting it all together

In [37]:
cond = (pets.Animal == 'Dog') & (pets.Gender == 'Female')
pets.loc[cond]

,Owner,Animal,Legs,Gender,Ownership_date
1,Carl,Dog,4,Female,11/6/19
10,Agnes,Dog,4,Female,11/11/22


## Task 5 - sorting
* To read a dataframe, it often helps to sort it
* Text columns are sorted lexicographically
* Number columns are sorted by value
* The methos is `sort_values()`

In [38]:
pets.sort_values('Animal')

,Owner,Animal,Legs,Gender,Ownership_date
3,Gabriel,Canari,2,NaN,3/1/23
2,John,Cat,4,Female,21/7/22
6,Brigitte,Cat,4,Female,13/4/20
0,Alfred,Dog,4,Male,14/5/21
1,Carl,Dog,4,Female,11/6/19
10,Agnes,Dog,4,Female,11/11/22
4,Harold,Hamster,4,Male,14/5/16
5,Anna,Iguana,4,NaN,29/8/22
7,Christine,Parrot,2,Male,15/9/19
9,Greta,Parrot,2,Male,17/7/18


In [39]:
pets.sort_values(['Legs', 'Animal'])

,Owner,Animal,Legs,Gender,Ownership_date
8,Eva,Snake,0,NaN,6/2/21
3,Gabriel,Canari,2,NaN,3/1/23
7,Christine,Parrot,2,Male,15/9/19
9,Greta,Parrot,2,Male,17/7/18
2,John,Cat,4,Female,21/7/22
6,Brigitte,Cat,4,Female,13/4/20
0,Alfred,Dog,4,Male,14/5/21
1,Carl,Dog,4,Female,11/6/19
10,Agnes,Dog,4,Female,11/11/22
4,Harold,Hamster,4,Male,14/5/16


### Reverse sorting order
* You can reverse the entire sort order with a 2nd parameter: `ascending=False`
* You can select which columns to sort in reverse order, by expanding the `ascending` parameter from a single value to a list

In [40]:
pets.sort_values(['Legs', 'Animal'], ascending=False)

,Owner,Animal,Legs,Gender,Ownership_date
5,Anna,Iguana,4,NaN,29/8/22
4,Harold,Hamster,4,Male,14/5/16
0,Alfred,Dog,4,Male,14/5/21
1,Carl,Dog,4,Female,11/6/19
10,Agnes,Dog,4,Female,11/11/22
2,John,Cat,4,Female,21/7/22
6,Brigitte,Cat,4,Female,13/4/20
7,Christine,Parrot,2,Male,15/9/19
9,Greta,Parrot,2,Male,17/7/18
3,Gabriel,Canari,2,NaN,3/1/23


In [41]:
pets.sort_values(['Legs', 'Animal'], ascending=[True, False])

,Owner,Animal,Legs,Gender,Ownership_date
8,Eva,Snake,0,NaN,6/2/21
7,Christine,Parrot,2,Male,15/9/19
9,Greta,Parrot,2,Male,17/7/18
3,Gabriel,Canari,2,NaN,3/1/23
5,Anna,Iguana,4,NaN,29/8/22
4,Harold,Hamster,4,Male,14/5/16
0,Alfred,Dog,4,Male,14/5/21
1,Carl,Dog,4,Female,11/6/19
10,Agnes,Dog,4,Female,11/11/22
2,John,Cat,4,Female,21/7/22


## Part 2 exrecise - see exercises notebook


### Skipping `loc`? recommendations
* Many times, you will see the `.loc` is skipped, e.g. `pets[cond]`
* *Most of the time*, it works as expected
* If you're getting unexpected behavior - rewrite using `loc`'s.

In [42]:
pets[cond]

,Owner,Animal,Legs,Gender,Ownership_date
1,Carl,Dog,4,Female,11/6/19
10,Agnes,Dog,4,Female,11/11/22


# Bonus - Date logic
**Which animals have been owned for more than 500 days?**
#### Step 1 - Convert date string to date representation
* We use `pd.to_datetime()` to convert a column
* We will assign the data back to the same column


In [ ]:
# dtypes Before
pets.dtypes

In [ ]:
pets['Ownership_date'] = pd.to_datetime(pets['Ownership_date'])
# dtypes After
pets.dtypes

#### Step 2 - store today's date in a variable

In [ ]:
import datetime
today = datetime.datetime(2023, 10, 19)

#### Step 3- Calculate time difference, convert to days

In [ ]:
(today - pets.Ownership_date).dt.days

#### Step 4: Add the information back to the data frame

In [ ]:
pets['Ownership_days'] = (today - pets.Ownership_date).dt.days

#### Step 5: query

In [ ]:
pets.query('Ownership_days > 500')